In [38]:
import pandas as pd
import numpy as np
import math

In [26]:
HISTORY = 6 #indicates how many past datapoints are considered (e.g prediction based on pas 6 hours)
FUTURE = 1  #indicates prediction into future (e.g 1 hour into the future)
OVERLAP = 3 #indicates how much the data is overlapped

In [27]:
data = pd.read_csv("1h_sampled_data.csv")

In [28]:
features = ['temp_pi', 'hum_pi', 'PM1_pi', 'PM2.5_pi', 'PM10_pi', 'wind_speed']

In [29]:
targets = ['PM2.5', 'PM10']

In [30]:
data = data[features]

In [97]:
data.iloc[220:280,:]

,temp_pi,hum_pi,PM1_pi,PM2.5_pi,PM10_pi,wind_speed
220,NaN,NaN,NaN,NaN,NaN,7.568566
221,NaN,NaN,NaN,NaN,NaN,6.618519
222,NaN,NaN,NaN,NaN,NaN,6.489992
223,NaN,NaN,NaN,NaN,NaN,8.049845
224,NaN,NaN,NaN,NaN,NaN,8.287822
225,NaN,NaN,NaN,NaN,NaN,5.600286
226,NaN,NaN,NaN,NaN,NaN,2.520000
227,NaN,NaN,NaN,NaN,NaN,6.924738
228,NaN,NaN,NaN,NaN,NaN,8.089993
229,NaN,NaN,NaN,NaN,NaN,1.297998


In [32]:
features_new = []
for i in range(HISTORY):
    features_new += [f + "_t" + str(i) for f in features]

In [35]:
features_new += targets

In [87]:
x = math.ceil(data.shape[0]/OVERLAP)
y = len(features)*HISTORY + len(targets)
data_new = np.zeros((x, y))

In [88]:
k = 0
for i in range(HISTORY - 1, data.shape[0], OVERLAP):
    temp = np.zeros(len(features_new))
    for j in range(HISTORY):
        temp[j*(HISTORY):(j+1)*(HISTORY)] = data.iloc[i - j, :]
    
    temp[HISTORY*HISTORY] = data.iloc[i+1, 3]
    temp[HISTORY*HISTORY+1] = data.iloc[i+1, 4]
    data_new[k,:] = temp
    k += 1

In [89]:
data_new = pd.DataFrame(data_new, columns=features_new)

In [90]:
data_new

,temp_pi_t0,hum_pi_t0,PM1_pi_t0,PM2.5_pi_t0,PM10_pi_t0,wind_speed_t0,temp_pi_t1,hum_pi_t1,PM1_pi_t1,PM2.5_pi_t1,...,PM10_pi_t4,wind_speed_t4,temp_pi_t5,hum_pi_t5,PM1_pi_t5,PM2.5_pi_t5,PM10_pi_t5,wind_speed_t5,PM2.5,PM10
0,17.625000,30.933333,4.365000,9.652500,15.223333,9.826088,19.983333,28.125000,4.610000,10.042500,...,15.388333,6.924738,8.316667,53.283333,6.042500,11.798333,17.710000,5.116561,6.641000,9.477000
1,11.616667,40.350000,2.710000,6.040000,9.553333,7.754637,12.683333,37.733333,4.529167,10.465000,...,15.750000,11.269782,18.650000,30.083333,3.752500,6.064167,8.284167,6.952755,7.147500,10.535000
2,8.466667,49.616667,4.839167,8.685833,12.543333,7.895416,9.416667,45.208333,3.826667,7.878333,...,16.762500,8.496304,13.990000,35.770000,3.787000,6.641000,9.477000,8.209263,8.777500,13.442500
3,6.516667,61.908333,5.355000,9.979167,14.669167,5.692100,6.850000,57.158333,5.519167,9.145833,...,12.093333,4.104631,10.425000,42.900000,3.815000,7.147500,10.535000,6.162207,7.956667,10.690833
4,4.981818,66.109091,6.809091,12.583636,18.430000,4.510787,5.358333,64.891667,6.549167,12.697500,...,12.672500,6.151683,7.566667,53.983333,4.285000,8.777500,13.442500,6.849467,13.833333,20.941667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,4.933333,76.922222,1.782222,4.942222,8.367778,15.137133,5.633333,77.383333,1.753333,5.618333,...,4.628000,12.074766,10.709091,77.600000,1.838182,4.190909,6.679091,16.935310,3.560000,4.923000
158,4.816667,78.983333,2.143333,4.403333,6.750833,14.618837,4.827273,78.490909,1.823636,4.080909,...,9.856667,18.171976,7.437500,73.687500,1.908750,5.385000,9.160000,21.995380,6.740000,10.450833
159,6.372727,72.300000,3.828182,8.692727,13.842727,13.684735,5.975000,75.625000,1.982500,3.617500,...,6.464545,12.574260,4.630000,78.930000,2.154000,3.560000,4.923000,8.534353,9.065833,13.411667
160,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [91]:
data_new.to_csv('model_data.csv')

In [95]:
data_new.iloc[70:75,:]

,temp_pi_t0,hum_pi_t0,PM1_pi_t0,PM2.5_pi_t0,PM10_pi_t0,wind_speed_t0,temp_pi_t1,hum_pi_t1,PM1_pi_t1,PM2.5_pi_t1,...,PM10_pi_t4,wind_speed_t4,temp_pi_t5,hum_pi_t5,PM1_pi_t5,PM2.5_pi_t5,PM10_pi_t5,wind_speed_t5,PM2.5,PM10
70,NaN,NaN,NaN,NaN,NaN,4.320000,NaN,NaN,NaN,NaN,...,NaN,3.600000,NaN,NaN,NaN,NaN,NaN,4.735060,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,7.771331,NaN,NaN,NaN,NaN,...,NaN,3.600000,NaN,NaN,NaN,NaN,NaN,3.671294,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,6.618519,NaN,NaN,NaN,NaN,...,NaN,8.534353,NaN,NaN,NaN,NaN,NaN,7.145796,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,8.287822,NaN,NaN,NaN,NaN,...,NaN,7.568566,NaN,NaN,NaN,NaN,NaN,6.877790,NaN,NaN
74,NaN,NaN,NaN,NaN,NaN,6.924738,NaN,NaN,NaN,NaN,...,NaN,8.049845,NaN,NaN,NaN,NaN,NaN,6.489992,NaN,NaN
